![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 2 - Clasificación de género de películas

El propósito de este proyecto es que puedan poner en práctica, en sus respectivos grupos de trabajo, sus conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 2: Clasificación de género de películas"

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 8. Sin embargo, es importante que avancen en la semana 7 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 8, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/2c54d005f76747fe83f77fbf8b3ec232).

## Datos para la predicción de género en películas

![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/moviegenre.png)

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros.

Agradecemos al profesor Fabio González, Ph.D. y a su alumno John Arevalo por proporcionar este conjunto de datos. Ver https://arxiv.org/abs/1702.01992

## Ejemplo predicción conjunto de test para envío a Kaggle
En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score, make_scorer, accuracy_score
from sklearn.model_selection import train_test_split

import re
import fasttext
import fasttext.util
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import nltk
from nltk.corpus import stopwords

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam, RMSprop
from keras import metrics
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.callbacks import EarlyStopping
from keras import backend as K
from livelossplot import PlotLossesKeras
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, ParameterGrid

import joblib

stop_words = set(stopwords.words('english'))
ft_model = fasttext.load_model('cc.en.300.bin')

In [24]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [25]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [26]:
# Visualización datos de test
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


## Preprocesamiento de texto

Se crea una función de preprocesameinto de texto, la cual convierte todo el texto en minúscula y elimina signos de puntuación que puedan intervenir en la tokenización de las palabras

In [27]:
#Se define la función

def preprocesamiento_texto(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [28]:
# Aplicar preprocesamiento

dataTraining['plot'] = dataTraining['plot'].apply(preprocesamiento_texto)
dataTraining['title'] = dataTraining['title'].apply(preprocesamiento_texto)

dataTesting['plot'] = dataTesting['plot'].apply(preprocesamiento_texto)
dataTesting['title'] = dataTesting['title'].apply(preprocesamiento_texto)

## Generar embeddings para los titulos y resumenes

Se crea una función que tokeniza el texto en cada palabra a través de la función .split() y se genera un embedding para cada palabra con fast text

**Aqui les dejo un resumen otorgado por el amigo del uso de Fast Text y porque lo seleccionamos para hacer los embeddings por encima de otros modelos como Word2Vec o Bert:**

//FastText es un modelo de aprendizaje profundo desarrollado por Facebook AI Research (FAIR) que se utiliza principalmente para tareas de procesamiento del lenguaje natural (PLN) como la clasificación de texto y la generación de embeddings de palabras. A continuación, se explica detalladamente su funcionamiento y por qué es especialmente útil en problemas de clasificación:

Funcionamiento de FastTe
xt
Representación de Palabras con N-grama:

A diferencia de otros modelos como Word2Vec, FastText descompone las palabras en sub-palabras o n-gramas de caracteres. Por ejemplo, la palabra "gato" podría descomponerse en los siguientes trigrama: <ga, gat, ato, to>.
Cada palabra se representa como la suma de los vectores de sus n-gramas, lo que permite que el modelo capte información morfológica y semántica a nivel sub-pa
labra.
Entrenamiento de Modelos:

Modelo Skip-Gram: Similar a Word2Vec, FastText utiliza el modelo Skip-Gram, donde la tarea es predecir el contexto de una palabra dada (las palabras que aparecen alrededor de la palabra objetivo en una ventana de co
ntexto).
Clasificación Jerárquica: Para tareas de clasificación de texto, FastText utiliza una estructura jerárquica de softmax, lo que hace más eficiente la predicción de categorías en un gran conjunto de e
tiquetas.
Construcción de Embeddings:

Los embeddings de palabras se generan a partir de los vectores de los n-gramas que componen las palabras. Esto permite que palabras similares en su forma (morfología) compartan información en sus embeddings.
Los embeddings de FastText pueden generalizar mejor para palabras fuera del vocabulario (OOV, por sus siglas en inglés) porque pueden construir representaciones para palabras no vistas previamente basándose en 
sus n-gramas.
Utilidad en Problemas de
 Clasificación
Manejo de Palabrs Raras y OOV:

Debido a la descomposición en n-gramas, FastText puede crear representaciones útiles para palabras raras o nuevas que no estaban presentes durante el entrenamiento. Esto es crucial en tareas de clasificación donde el vocabulario puede ser muy va
riado y dinámico.
Mejora en la Represetación Semántica:

La consideración de sub-palabras permite que FastText capture relaciones morfológicas y semánticas entre palabras, lo que mejora la calidad de los embeddings. Esto, a su vez, se traduce en una mejor capacidad de generalización en tareas de cla
sificación de texto.
Eficencia Computacional:

FastText es altamente eficiente en términos de tiempo y recursos computacionales tanto en el entrenamiento como en la inferencia. Utiliza una aproximación jerárquica para reducir el costo computacional en tareas de clasificación
 con muchas categorías.
Adaptabilidad a Diferenes Lenguas y Dialectos:

El uso de n-gramas hace que FastText sea especialmente robusto para lenguas con rica morfología o múltiples dialectos, donde las variaciones en
 las palabras son comunes.
Aplicabildad a Tareas Multilingües:

Los embeddings de FastText pueden ser entrenados en múltiples lenguas, lo que permite aplicaciones multilingües sin necesidad de modelos separados para cada idioma. Esto es especialmente útil en aplicaciones globales que requieren
 soporte para varios idio
mas.
Ejemplos de Aplicacione
Clasificación de Documentos:

FastText se utiliza para clasificar grandes volúmenes de texto en categorías predefinidas, como la clasificación de noticias, revisión de produ
ctos, y análisis desentimientos.
Detección de Spam:

En sistemas de correo electrónico o redes sociales, FastText puede ayudar a clasificar y filtrar 
mensajes de spam con alta pecisión.
Sistemas de Recomendación:

Al comprender las preferencias de los usuarios a través del análisis de texto (por ejemplo, reseñas de productos), FastText puede mejo
rar las recomendaciones pFonalizadas.
Análisis de Sentimientos:

FastText es utilizado para detectar y clasificar opiniones en texto, por ejemplo, para monitorear las reds
 sociales o analizar reseñas de productos.
En resumen, FastText es un modelo poderoso y eficiente para generar embeddings y realizar clasificación de texto, especialmente útil por su capacidad para manejar palabras raras y nuevas a través del uso de// n-gramas y por su eficiencia computacional.e n-gramas y por su eficiencia computacional.

In [29]:
# Función para obtener los embeddings a partir de Fast text

def obtener_embedding(texto, modelo):
    tokens = texto.split()
    embeddings = [modelo.get_word_vector(word) for word in tokens]
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(modelo.get_dimension())

In [30]:
# Transformación de variables

X_plot = np.array([obtener_embedding(resumen, ft_model) for resumen in dataTraining['plot']])
X_title = np.array([obtener_embedding(titulo, ft_model) for titulo in dataTraining['title']])

scaler = StandardScaler()
scaler.fit(dataTraining[['year']])

X_year = scaler.transform(dataTraining[['year']])

X = np.concatenate([X_year, X_title, X_plot], axis=1)

dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

## Definición y entrenamiento del modelo (Red Neuronal)

Aqui se entrena la red Neuronal con los conjuntos X_train y y_train_genres. Queda pendiente calibrar para cargar el modelo, pero este modelo 'Lite' tiene mejor desempeño que el que estaba propuesto

In [31]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X, y_genres, test_size=0.33, random_state=42)

In [32]:
clf = Sequential()
clf.add(Dense(X.shape[1], input_dim=X.shape[1], activation='relu'))
clf.add(Dense(256, activation='relu'))
clf.add(Dense(y_genres.shape[1], activation='sigmoid'))  # Sigmoid para multietiqueta

clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
clf.fit(X_train, y_train_genres, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1845 - loss: 0.3880 - val_accuracy: 0.2429 - val_loss: 0.2648
Epoch 2/30
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3236 - loss: 0.2458 - val_accuracy: 0.3242 - val_loss: 0.2290
Epoch 3/30
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3721 - loss: 0.2094 - val_accuracy: 0.3355 - val_loss: 0.2157
Epoch 4/30
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3876 - loss: 0.1947 - val_accuracy: 0.3667 - val_loss: 0.2115
Epoch 5/30
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4182 - loss: 0.1813 - val_accuracy: 0.3450 - val_loss: 0.2086
Epoch 6/30
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4246 - loss: 0.1688 - val_accuracy: 0.3771 - val_loss: 0.2076
Epoch 7/30
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4328 - loss: 0.1595 - val_accuracy: 0.3866 - val_loss: 0.2086
Epoch 8/30
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4490 - loss: 0.1495 - val_accuracy: 0.

In [33]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step


0.8475369125080784

## Generar archivos Plk

In [34]:
joblib.dump(clf, './model_deployment/red_neuronal_peliculas.pkl', compress=3)
joblib.dump(scaler, './model_deployment/scaler_year.pkl', compress=3)

['./model_deployment/scaler_year.pkl']

## Función para predecir probabilidad genero películas

Aqui se define una función que retorna un diccionario con las probabilidades de cada película de pertenecer a cada genero usando los modelos creados y las funciones de obtener embeddings y preprocesamiento de texto construidas anteriormente. En la última celda existe una prueba de funcionamiento 

In [35]:
# Se define la función
def probabilidad_genero_pelicula(year, title, plot):

    #scaler = StandardScaler()

    dicc = {'year': [year],
            'title': [title],
            'plot': [plot]}
    
    base = pd.DataFrame(dicc)

    base['plot'] = base['plot'].apply(preprocesamiento_texto)
    base['title'] = base['title'].apply(preprocesamiento_texto)

    X_plot = np.array([obtener_embedding(resumen, ft_model) for resumen in base['plot']])
    X_title = np.array([obtener_embedding(titulo, ft_model) for titulo in base['title']])

    X_year = scaler.transform(base[['year']])

    X = np.concatenate([X_year, X_title, X_plot], axis=1)
    
    cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']
    
    #clf = red_neuronal
    
    proba_genres = clf.predict(X)
    df = pd.DataFrame(proba_genres, columns=cols)
    diccionario = df.to_dict(orient='list')

    return diccionario

In [36]:
# Prueba de funcionamiento
probabilidad_genero_pelicula('1999', 'Drugs', 'This is a comedy movie')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


{'p_Action': [1.285593123071381e-16],
 'p_Adventure': [1.66012738495383e-13],
 'p_Animation': [3.771059908785901e-08],
 'p_Biography': [1.5381954654003493e-05],
 'p_Comedy': [1.0],
 'p_Crime': [3.026908501624348e-08],
 'p_Documentary': [0.24686846137046814],
 'p_Drama': [8.113729563774541e-07],
 'p_Family': [2.6588822810147406e-10],
 'p_Fantasy': [1.1559663274339493e-13],
 'p_Film-Noir': [1.5108414255909172e-22],
 'p_History': [3.4666839304796707e-15],
 'p_Horror': [8.648400944346479e-17],
 'p_Music': [6.006782768963603e-07],
 'p_Musical': [3.0062039968470344e-06],
 'p_Mystery': [2.914296581835174e-10],
 'p_News': [2.498630208985775e-12],
 'p_Romance': [0.007413754239678383],
 'p_Sci-Fi': [3.4234014778667553e-19],
 'p_Short': [1.159564999397844e-05],
 'p_Sport': [1.0731306048319435e-14],
 'p_Thriller': [1.3025129722202066e-19],
 'p_War': [3.0382564953496116e-12],
 'p_Western': [3.379485435510307e-16]}

## Busqueda de hiperparámetros

En esta parte estoy buscando un método que nos permita calibrar todos los hiper parámetros que contiene el diccionario param_grid. Ya que el KerasClassifier no admite todos los hiperparámetros para iterar a traves de Grid Search o random Search.


Tener en cuenta que estas funciones y códigos de esta sección hacia abajo **NO** van en el archivo py para montaje en AWS

In [37]:
# Define tu función para construir el modelo
def build_model(optimizer='adam', init='glorot_normal', dropout_rate=0.2, neurons=350, neurons2=100):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X.shape[1], kernel_initializer=init, activation='sigmoid'))
    model.add(Dense(neurons2, activation='sigmoid'))
    model.add(Dropout(dropout_rate, input_shape=(X.shape[1],)))
    model.add(Dense(y_genres.shape[1], kernel_initializer=init, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [38]:
param_grid = {
    'optimizer': ['rmsprop', 'adam'],
    'init': ['glorot_uniform', 'normal', 'he_normal'],
    'epochs': [50, 100, 150, 200],
    'batch_size': [5, 10, 20, 25],
    'dropout_rate': [0.0, 0.1, 0.2, 0.3],
    'neurons': [200, 350, 500],
    'neurons2':[100, 200, 300]
}


## Generar predicciones sobre el conjunto test para cargar a la competencia

Se deja todo marcado como markdown hasta calibrar el modelo para poder ejecutar todo el codigo de ser necesario

"# transformación variables predictoras X del conjunto de test"
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

"# Predicción del conjunto de test
y_pred_test_genres = clf.predict_proba(X_test_dtm)

"# Guardar predicciones en formato exigido en la competencia de kaggle"
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_RF.csv', index_label='ID')
res.head()